In [ ]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
!pip install probscale
import pywt
import matplotlib.pyplot as plt
import statsmodels.api as sm  
from datetime import datetime
import math
import scipy as sp
import sklearn
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas_profiling as pp
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import stats
import numpy
import probscale
import seaborn
import math
from math import log
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import datetime
from pandas.plotting import lag_plot
from matplotlib import style
%matplotlib inline
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

!pip install pyforest
from pyforest import *
import pandas_datareader as web
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
plt.style.use('ggplot')
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# %% [code]
pip install --upgrade pip

# %% [code]
data = pd.read_excel("/kaggle/input/natural-gas/Natutalgas.xlsx", parse_dates=True)
data.head()

# %% [code]
data = data.rename(columns=data.iloc[0]).drop(data.index[0])
data.head()

# %% [code]
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
df = data.set_index(data.columns[0])
df = df.sort_index(ascending=True)
df.head()

# %% [code]
plt.style.use('dark_background')
df['hh_sp'].plot(figsize = (15,5), grid=True)
plt.title('Henry Hub Spot Price')
plt.ylabel('Price(Dollars per Million Btu)')
plt.show()

# %% [code]
plt.style.use('dark_background')
# Plot
fig, axes = plt.subplots(1, 4, figsize=(10,3), sharex=True, sharey=True, dpi=100)
for i, ax in enumerate(axes.flatten()[:4]):
    lag_plot(df, lag=i+1, ax=ax)
    ax.set_title('Lag ' + str(i+1))

fig.suptitle('Lag Plots of Natural Gas \n(Points get wide and scattered with increasing lag -> lesser correlation)\n', y=1.15)    
plt.show()

# %% [code]
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df.hh_sp)
plt.show()

# %% [code]
df.dtypes

# %% [code]
df = df.astype('float')
df.info()

# %% [code]
df['return'] = df['hh_sp'].pct_change() * 100
df.shape

# %% [code]
plt.plot(df['return'])

# %% [code]
plt.figure()
plt.subplot(211)
plot_acf(df['hh_sp'], ax=plt.gca())
plt.subplot(212)
plot_pacf(df['hh_sp'], ax=plt.gca())
plt.xlabel('Lag')
plt.tight_layout()
plt.show()

# %% [code]
np.isnan(df).sum()

# %% [code]
def fill_missing(df):
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(df[row,col]):
                df[row,col]= df[row-1, col]

fill_missing(df.values)
np.isnan(df).sum()

# %% [code]
fill_missing(df.values)
np.isnan(df).sum()

# %% [code]
import seaborn as sns
sns.despine(left=True)

# Plot a simple histogram with binsize determined automatically
sns.distplot(df['hh_sp'], color="b")
plt.title('Henry hub spot price')
plt.grid(True)
plt.show()

# %% [code]
from scipy import stats
stats.jarque_bera(df['hh_sp'])

# %% [code]
fig, ax = plt.subplots(figsize=(10,4))
plt.grid(True)
fig = probscale.probplot(df['hh_sp'], ax=ax, plottype='pp', bestfit=True,
                         problabel='Percentile', datalabel='HenryHub spot price (S/MMBtu)',
                         scatter_kws=dict(label='HH original data'),
                         line_kws=dict(label='Best-fit line'))
ax.legend(loc='upper left')
seaborn.despine()

# %% [code]
fig, ax = plt.subplots(figsize=(10,4))
plt.grid(True)
fig = probscale.probplot(df['hh_sp'], ax=ax, plottype='pp', bestfit=True,
                         datascale='log',
                         problabel='Percentile', 
                         datalabel='Log of HenryHub price ($/MMBtu)',
                         scatter_kws=dict(label='HH lognormal data'),
                         line_kws=dict(label='Best-fit line'))
ax.legend(loc='upper left')
seaborn.despine()

# %% [code]
df.columns

# %% [code]
df.tail(2)

# %% [markdown]
# - Target variable is +1 if the spot price tomorrow is higher than the spot price today, and -1 if the spot price tomorrow is lower than the spot price today.
# - we assume that the spot price tomorrow is not the same as the spot price today, which we can choose to handle by creating a third categorical value, 0.

# %% [code]
dclas = df.copy()
dclas = dclas.loc[:, ~ dclas.columns.duplicated()] # drop duplicated columns if any
dclas['target'] = np.where(dclas['hh_sp'].shift(-1) > dclas['hh_sp'], 1, -1)
dclas.dropna(inplace=True)
# classification set

lags = 3
# Create the shifted lag series of prior trading period close values
for i in range(0, lags):
    dclas["Lag%s" % str(i+1)] = dclas["hh_sp"].shift(i+1).pct_change()
dclas = dclas.dropna()
x = dclas.drop(['hh_sp', 'target'], 1)
y_clas = dclas.target

# %% [code]
# Checking Correlation 
sns.set(style='darkgrid', context='talk', palette='Dark2')
plt.figure(figsize=(14,5))
dclas.corr()['target'].sort_values(ascending = False).plot(kind='bar')
plt.show()

# %% [code]
import seaborn as sns
sns.countplot(x = 'target', data=dclas, hue='target')
plt.show()


# %% [code]
from sklearn.preprocessing import StandardScaler
#Initialize and fit scaler
scaler = StandardScaler()
#Fit scaler using the training data
scaler.fit(x)

#Transform the raw data
x_st = scaler.transform(x)
print("Number of features: ", x_st.shape[1])

# %% [code]
from sklearn.feature_selection import RFECV,SelectPercentile,mutual_info_classif, SelectKBest, f_classif
# FEATURE SELECTION
#1
#y = y_clas
mi_select = SelectPercentile(mutual_info_classif, 
                             percentile=60).fit(x_st,  np.ravel(y_clas))
# remove lower 40%
x_mi = mi_select.transform(x_st)
print("Feature Selection Results:")
print("Filter Result:"); print("Number of features: ", x_mi.shape[1])

# %% [code]
#2
# Corr_threshold is the cuttoff level for correlation coefficient
def correlation(x_tr, corr_threshold):
    corr = x.corr(method = "spearman").abs()
    
    """Spearman’s correlation is non-parametric and does not assume a linear relationship between variables; 
    it looks for monotonic relationships"""

    upper = corr.where(np.triu(np.ones(corr.shape), k = 1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > corr_threshold)]
    x_corr_filtered = x.drop(to_drop, axis = 1)
    return x_corr_filtered

def heat_map(corr,title):
    sns.set(font_scale = 1.0)
    f, ax = plt.subplots(figsize=(11, 9))
    sns.heatmap(corr, cmap= "YlGnBu", square=True, ax = ax)
    f.tight_layout()
    ax.set_title(title)

mi = pd.DataFrame(x_mi,
                     index = x.index,
                     columns = x.columns[mi_select.get_support()])

x_corr = correlation(mi, 0.95)

"""Spearman correlation coefficient is based on the ranked values for each variable""" 
corr_before = mi.corr(method = "spearman").abs()
corr_after = x_corr.corr(method = "spearman").abs()

heat_map(corr_before, "Heat Map (BEFORE FILTERING)")
heat_map(corr_after, "Heat Map (AFTER FILTERING)")

print("Correlation Filter Result:")
print("Number of features: ", x_corr.shape[1])

# %% [code]
!pip install pca
from pca import pca
from sklearn.decomposition import PCA

# %% [code]
#3
#PCA
plt.style.use('seaborn-whitegrid')
# Initialize to reduce the data up to the number of componentes that explains 99% of the variance.
model = pca(n_components = 0.99)
# Fit transform
x_pca = model.fit_transform(x_corr)
# Plot explained variance
fig, ax = model.plot()

tot_var = 0.99 # total variance
x_pca = PCA(tot_var, svd_solver = 'full').fit(x_corr)
print(x_pca.explained_variance_ratio_); print()
print("Reduced dimensions can explain {:.4f}".format(sum(x_pca.explained_variance_ratio_)),
      "% of the variance in the original data"); print()
print(x_pca.components_.shape[0]); print()
# number of components
n_pcs= x_pca.components_.shape[0]
# get the index of the most important feature on EACH component
most_important = [np.abs(x_pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = x_corr.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
p= pd.DataFrame(dic.items())
print(p); print()

pca = PCA(tot_var, svd_solver = 'full')
x_pca = pca.fit_transform(x_corr)


# %% [code]
from sklearn.model_selection import train_test_split
SplitFactor = 0.8
xTrain, xTest, yTrain, yTest = train_test_split(x_pca, y_clas, shuffle=False,
                                                train_size = SplitFactor)

# %% [code]
# Create models
print("Accuracy scores/Confusion Matrices:\n")
models = [("LR", LogisticRegression()),
          ("LDA", LinearDiscriminantAnalysis()),
          ("QDA", QuadraticDiscriminantAnalysis()),
          ("LSVC", LinearSVC()),
          ("RSVM", SVC(C=1000000.0, cache_size=200, class_weight=None,
                       coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
                       max_iter=-1, probability=False, random_state=None,
                       shrinking=True, tol=0.001, verbose=False)),
          ("RF", RandomForestClassifier(
              n_estimators=100, criterion='gini',
              max_depth=None, min_samples_split=2,
              min_samples_leaf=1, max_features='auto',
              bootstrap=True, oob_score=False, n_jobs=1,
              random_state=None, verbose=0))]
# iterate over the models
for m in models:
    # Train each of the models on the training set
    m[1].fit(xTrain, yTrain)
    # predictions on the test set
    pred = m[1].predict(xTest)
    # Accuracy Score and the confusion matrix for each model
    print("%s:\n%0.3f" % (m[0], m[1].score(xTest, yTest)))
    print("%s\n" % confusion_matrix(pred, yTest))

# %% [code]
rfc = RandomForestClassifier(
              n_estimators=100, criterion='gini',
              max_depth=None, min_samples_split=2,
              min_samples_leaf=1, max_features='auto',
              bootstrap=True, oob_score=False, n_jobs=1,
              random_state=None, verbose=0).fit(xTrain, yTrain)

pd.set_option('float_format', '{:f}'.format)
test_pred = rfc.predict(xTest)
print('Test prediction values:')
test_pred = pd.DataFrame(test_pred)
test_pred.rename(columns = {0: 'TestPrediction'}, inplace=True); 
test_pred.index = yTest.index
compare = pd.concat([yTest, test_pred], 1)
print(compare)

# %% [code]
yTest.shape

# %% [code]
# collect last 100 positions for visualization
dd = pd.DataFrame(df.hh_sp[-45:])
combine = pd.concat([dd, compare],1)
combine

# %% [code]
print('Actual value counts:')
print(compare['target'].value_counts())
print('Predicted value counts:')
print(compare['TestPrediction'].value_counts())

# %% [markdown]
# ## Regression

# %% [code]
dreg = df.copy()
dreg['target'] = dreg['hh_sp'].shift(-1) - dreg['hh_sp']
dreg.dropna(inplace=True)
dreg.tail() 

# %% [code]
dreg.shape

# %% [code]
lags = 3
# Create the shifted lag series of prior trading period close values
for i in range(0, lags):
    dreg["Lag%s" % str(i+1)] = dreg["hh_sp"].shift(i+1).pct_change()
dreg = dreg.dropna()
x_reg = dreg.drop(['hh_sp', 'target'],1)
y_reg = dreg.target

# %% [code]
print(x_reg.shape); print(y_reg.shape)

# %% [code]
from pca import pca
from sklearn.decomposition import PCA

# %% [code]
#Initialize and fit scaler
scaler = StandardScaler()
#Fit scaler using the training data
scaler.fit(x_reg)

#Transform the raw data
xScaler = scaler.transform(x_reg)
print("Number of original features: ", xScaler.shape[1])

from sklearn.feature_selection import mutual_info_regression
MiSelect = SelectPercentile(mutual_info_regression, 
                             percentile=60).fit(xScaler,y_reg.values)
# 1 (mutual information)
xMi = MiSelect.transform(xScaler)
print("Mutual information Feature Selection Results:")
print("Filter Result:"); print("Number of features: ", xMi.shape[1])

MI = pd.DataFrame(xMi,
                     index = x_reg.index,
                     columns = x_reg.columns[MiSelect.get_support()])

# 2 (correlation)
# Corr_threshold is the cuttoff level for correlation coefficient
def correlation(x_reg, corr_threshold):
    corr = x_reg.corr(method = "spearman").abs()
    
    """Spearman’s correlation is non-parametric and does not assume a linear relationship between variables; 
    it looks for monotonic relationships"""

    upper = corr.where(np.triu(np.ones(corr.shape), k = 1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > corr_threshold)]
    x_corr_filtered = x_reg.drop(to_drop, axis = 1)
    return x_corr_filtered

Mi = pd.DataFrame(xMi,
                     index = x_reg.index,
                     columns = x_reg.columns[mi_select.get_support()])

xCorr = correlation(Mi, 0.95)
print("Correlation Filter Result:")
print("Number of features: ", xCorr.shape[1])


# 3 (principal component analysis)
tot_var = 0.99 # total variance
pca = PCA(tot_var, svd_solver = 'full')
xPca = pca.fit_transform(xCorr)

from sklearn.model_selection import train_test_split
SplitFactor = 0.8
xTrain, xTest, yTrain, yTest = train_test_split(xPca, y_reg, shuffle=False,
                                                train_size = SplitFactor)

rfc = RandomForestRegressor().fit(xTrain, yTrain)
pd.set_option('float_format', '{:f}'.format)
test_pred = rfc.predict(xTest)

# The mean squared error
print("Mean squared error (train): %.2f"% mean_squared_error(yTrain, 
                                                            rfc.predict(xTrain)))
# Explained variance score: 1 is perfect prediction
print('Variance score (train): %.2f' % r2_score(yTrain, 
                                                rfc.predict(xTrain)))
# The mean squared error
print("Mean squared error (test): %.2f" % mean_squared_error(yTest, 
                                                             rfc.predict(xTest)))
# Explained variance score: 1 is perfect prediction
print('Variance score (test): %.2f' % r2_score(yTest, rfc.predict(xTest)))

# %% [markdown]
# R^2 value close to 1 means better model fit, and the closer the value to 0, the worse the fit. Negative values mean that the regression model fits worse than the baseline model. Models with negative R^2 values usually indicate issues in the training data or process and cannot be used.